In [7]:
import cv2
import numpy as np

In [8]:
min_contour_width = 40  
min_contour_height = 40  
offset = 10  
line_height = 550  
matches = []
vehicles = 0

In [9]:
def get_centrolid(x, y, w, h):
   x1 = int(w / 2)
   y1 = int(h / 2)
 
   cx = x + x1
   cy = y + y1
   return cx, cy

In [10]:
cap = cv2.VideoCapture('Video2.mp4')

In [11]:
cap.set(3, 1920)
cap.set(4, 1080)
 
if cap.isOpened():
   ret, frame1 = cap.read()
else:
   ret = False
ret, frame1 = cap.read()
ret, frame2 = cap.read()

In [12]:
while ret:
   d = cv2.absdiff(frame1, frame2)
   grey = cv2.cvtColor(d, cv2.COLOR_BGR2GRAY)
 
   blur = cv2.GaussianBlur(grey, (5, 5), 0)
 
   ret, th = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
   dilated = cv2.dilate(th, np.ones((3, 3)))
   kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
 
  
   closing = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, kernel)
   contours, h = cv2.findContours(
       closing, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
   for(i, c) in enumerate(contours):
       (x, y, w, h) = cv2.boundingRect(c)
       contour_valid = (w >= min_contour_width) and (
           h >= min_contour_height)
 
       if not contour_valid:
           continue
       cv2.rectangle(frame1, (x-10, y-10), (x+w+10, y+h+10), (255, 0, 0), 2)
 
       cv2.line(frame1, (0, line_height), (1200, line_height), (0, 255, 0), 2)
       centrolid = get_centrolid(x, y, w, h)
       matches.append(centrolid)
       cv2.circle(frame1, centrolid, 5, (0, 255, 0), -1)
       cx, cy = get_centrolid(x, y, w, h)
       for (x, y) in matches:
           if y < (line_height+offset) and y > (line_height-offset):
               vehicles = vehicles +1
               matches.remove((x, y))
 
   cv2.putText(frame1, "Total Vehicle Detected: " + str(vehicles), (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1,
               (0, 170, 0), 2)
 
 
 
   cv2.imshow("Vehicle Detection", frame1)
   if cv2.waitKey(20) == 27:
       break
   frame1 = frame2
   ret, frame2 = cap.read()
 
cv2.destroyAllWindows()
cap.release()